In [5]:
#**********GET THE DATASET*************
#If the model is already trained you don't need to do this 
from roboflow import Roboflow
rf = Roboflow(api_key="L92yodulFT4LlHfg1I5l")
project = rf.workspace("aiclimbing").project("hold-detector")
dataset = project.version(13).download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Hold-Detector-13 in yolov8:: 100%|███████████████| 214/214 [00:00<00:00, 2058.87it/s]


In [6]:
#*********TRAIN THE MODEL**********
#This should already be pretrained so you don't need to do this either
#also it takes like an hour
from ultralytics import YOLO 
import os
# Training.
model = YOLO("yolov8n.pt")
try:
    os.rename("Hold-Detector-13", "datasets")
except:
    pass
try:
    os.rename("datasets/data.yaml", "data.yaml")
except:
    pass
results = model.train(
    data='data.yaml',
    imgsz=1280,
    epochs=50,
    batch=8,
    name='yolov8n_v8_50e',
)

New https://pypi.org/project/ultralytics/8.0.221 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196  Python-3.8.18 torch-2.1.1 CPU (Intel Core(TM) i7-10875H 2.30GHz)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=50, patience=50, batch=8, imgsz=1280, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8n_v8_50e, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnosti


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      25/50         0G      1.288     0.9263      1.192        102       1280: 100%|██████████| 12/12 [02:09<00:00, 10.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:02<0
                   all          4        107      0.861      0.927      0.913      0.548

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      26/50         0G      1.363     0.9639      1.225        122       1280: 100%|██████████| 12/12 [02:05<00:00, 10.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:03<0
                   all          4        107      0.867      0.917      0.924      0.554

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      27/50         0G      1.336     0.8841      1.183        168       1280: 100%|██████████| 12/12 [02

      45/50         0G      1.185     0.8903      1.168         54       1280: 100%|██████████| 12/12 [02:01<00:00, 10.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:02<0
                   all          4        107      0.864      0.951      0.935      0.604

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      46/50         0G      1.167     0.7761      1.128        172       1280: 100%|██████████| 12/12 [02:02<00:00, 10.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:02<0
                   all          4        107       0.88      0.925      0.926      0.598

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      47/50         0G      1.198       0.84      1.148        152       1280: 100%|██████████| 12/12 [02:01<00:00, 10.
                 Class     Images  Instances      Box(P         

In [65]:
from PIL import Image
import numpy as np
from ultralytics import YOLO
from ultralytics.engine.results import Boxes
import cv2
# Load a pretrained YOLOv8n model
model = YOLO('runs/detect/yolov8n_v8_50e22/weights/best.pt')

# Gets the image at the specified path as a numpy array
def getImage(path):
    img = cv2.imread(path)
    return img

# Displays the results of model(image)
def dispResults(results):
    for r in results:
        im_array = r.plot()  # plot a BGR numpy array of predictions
        im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
        im.show()  # show image
        im.save('results.jpg')  # save image

# Removes all bounding boxes in the results of model(image)
# that may be cut off by the sides
def removeEdges(results):
    for r in results:
        boxes_coords = np.array(r.boxes.xyxy)
        boxes_last = np.array(r.boxes.data[:, 4:])
        i=0
        while i < (len(boxes_coords)):
            x1, y1, x2, y2 = boxes_coords[i]
            imh = r.boxes.orig_shape[0]
            imw = r.boxes.orig_shape[1]
            if (x1 < imw / 100.0) or (y1 < imh / 100) or (x2 > imw - imw / 100.0) or (y2 > imh - imh / 100.0):
                boxes_coords = np.delete(boxes_coords, i, 0)
                boxes_last = np.delete(boxes_last, i, 0)
                i -= 1
            i += 1
        new_data = np.concatenate((boxes_coords, boxes_last), axis=1)
        newboxes = Boxes(new_data, r.boxes.orig_shape)
        r.boxes = newboxes
    return results


In [73]:
import os

def getBoxAsImage(results, index):
    r = results[0]
    x1, y1, x2, y2 = np.array(r.boxes.xyxy)[index]
    im_array = np.array(r.orig_img)
    cropped = im_array[round(y1):round(y2), round(x1):round(x2)]
    return cropped
        
def individualHolds(model, img_path, hold_path):
    try:
        os.rmdir(hold_path)
    except:
        pass
    os.mkdir(hold_path)
    for img_name in os.listdir(img_path):   
        img = getImage(f"{img_path}/{img_name}")
        results = model(img)
        results = removeEdges(results)
        r = results[0]
        for i in range(len(r.boxes)):
            im_array = getBoxAsImage(results, i)
            im = Image.fromarray(im_array)
            im.save(f"{hold_path}/{str(i) + '-'}{img_name}")

In [ ]:
individualHolds(model, "wall-images", "holds")


0: 1280x960 1 hold, 198.4ms
Speed: 13.1ms preprocess, 198.4ms inference, 1.0ms postprocess per image at shape (1, 3, 1280, 960)

0: 1280x960 2 holds, 183.7ms
Speed: 12.1ms preprocess, 183.7ms inference, 1.0ms postprocess per image at shape (1, 3, 1280, 960)

0: 1280x960 3 holds, 173.2ms
Speed: 11.9ms preprocess, 173.2ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 960)

0: 1280x960 3 holds, 183.2ms
Speed: 12.5ms preprocess, 183.2ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 960)

0: 1280x960 3 holds, 179.8ms
Speed: 12.0ms preprocess, 179.8ms inference, 1.0ms postprocess per image at shape (1, 3, 1280, 960)

0: 1280x960 11 holds, 185.5ms
Speed: 12.0ms preprocess, 185.5ms inference, 1.0ms postprocess per image at shape (1, 3, 1280, 960)

0: 1280x960 8 holds, 177.5ms
Speed: 12.0ms preprocess, 177.5ms inference, 1.0ms postprocess per image at shape (1, 3, 1280, 960)

0: 1280x960 1 hold, 175.3ms
Speed: 12.9ms preprocess, 175.3ms inference, 1.0ms postproces


0: 1280x960 2 holds, 186.3ms
Speed: 13.0ms preprocess, 186.3ms inference, 1.0ms postprocess per image at shape (1, 3, 1280, 960)

0: 1280x960 3 holds, 215.8ms
Speed: 13.4ms preprocess, 215.8ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 960)

0: 1280x960 2 holds, 178.9ms
Speed: 14.0ms preprocess, 178.9ms inference, 1.0ms postprocess per image at shape (1, 3, 1280, 960)

0: 1280x960 6 holds, 183.1ms
Speed: 12.0ms preprocess, 183.1ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 960)

0: 1280x960 6 holds, 186.3ms
Speed: 11.9ms preprocess, 186.3ms inference, 1.0ms postprocess per image at shape (1, 3, 1280, 960)

0: 1280x960 3 holds, 196.7ms
Speed: 12.5ms preprocess, 196.7ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 960)

0: 1280x960 5 holds, 185.4ms
Speed: 12.0ms preprocess, 185.4ms inference, 1.0ms postprocess per image at shape (1, 3, 1280, 960)

0: 1280x960 6 holds, 202.8ms
Speed: 14.5ms preprocess, 202.8ms inference, 1.0ms postproce

0: 1280x960 4 holds, 194.6ms
Speed: 13.5ms preprocess, 194.6ms inference, 1.0ms postprocess per image at shape (1, 3, 1280, 960)

0: 1280x960 3 holds, 196.1ms
Speed: 12.0ms preprocess, 196.1ms inference, 1.0ms postprocess per image at shape (1, 3, 1280, 960)

0: 1280x960 4 holds, 189.3ms
Speed: 12.0ms preprocess, 189.3ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 960)

0: 1280x960 3 holds, 194.1ms
Speed: 12.9ms preprocess, 194.1ms inference, 1.0ms postprocess per image at shape (1, 3, 1280, 960)

0: 1280x960 3 holds, 190.9ms
Speed: 14.0ms preprocess, 190.9ms inference, 1.1ms postprocess per image at shape (1, 3, 1280, 960)

0: 1280x960 2 holds, 196.9ms
Speed: 12.0ms preprocess, 196.9ms inference, 1.0ms postprocess per image at shape (1, 3, 1280, 960)

0: 1280x960 2 holds, 196.6ms
Speed: 12.0ms preprocess, 196.6ms inference, 1.0ms postprocess per image at shape (1, 3, 1280, 960)

0: 1280x960 3 holds, 185.1ms
Speed: 12.0ms preprocess, 185.1ms inference, 1.0ms postproces


0: 1280x960 3 holds, 198.0ms
Speed: 13.0ms preprocess, 198.0ms inference, 1.5ms postprocess per image at shape (1, 3, 1280, 960)

0: 1280x960 8 holds, 210.1ms
Speed: 13.0ms preprocess, 210.1ms inference, 1.0ms postprocess per image at shape (1, 3, 1280, 960)

0: 1280x960 3 holds, 222.2ms
Speed: 13.4ms preprocess, 222.2ms inference, 1.0ms postprocess per image at shape (1, 3, 1280, 960)

0: 1280x960 3 holds, 200.1ms
Speed: 13.0ms preprocess, 200.1ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 960)

0: 1280x960 9 holds, 171.7ms
Speed: 10.9ms preprocess, 171.7ms inference, 3.0ms postprocess per image at shape (1, 3, 1280, 960)

0: 1280x960 8 holds, 182.8ms
Speed: 14.4ms preprocess, 182.8ms inference, 1.0ms postprocess per image at shape (1, 3, 1280, 960)

0: 1280x960 2 holds, 191.8ms
Speed: 13.0ms preprocess, 191.8ms inference, 1.0ms postprocess per image at shape (1, 3, 1280, 960)

0: 1280x960 1 hold, 201.0ms
Speed: 12.0ms preprocess, 201.0ms inference, 1.0ms postproces

Speed: 13.0ms preprocess, 196.1ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 960)

0: 1280x960 (no detections), 193.2ms
Speed: 11.9ms preprocess, 193.2ms inference, 0.0ms postprocess per image at shape (1, 3, 1280, 960)

0: 1280x960 (no detections), 199.8ms
Speed: 12.9ms preprocess, 199.8ms inference, 1.0ms postprocess per image at shape (1, 3, 1280, 960)

0: 1280x960 2 holds, 194.8ms
Speed: 12.2ms preprocess, 194.8ms inference, 1.0ms postprocess per image at shape (1, 3, 1280, 960)

0: 1280x960 (no detections), 169.1ms
Speed: 12.4ms preprocess, 169.1ms inference, 1.0ms postprocess per image at shape (1, 3, 1280, 960)

0: 1280x960 1 hold, 182.4ms
Speed: 12.1ms preprocess, 182.4ms inference, 1.0ms postprocess per image at shape (1, 3, 1280, 960)

0: 1280x960 4 holds, 188.9ms
Speed: 12.0ms preprocess, 188.9ms inference, 1.0ms postprocess per image at shape (1, 3, 1280, 960)

0: 1280x960 2 holds, 194.2ms
Speed: 12.1ms preprocess, 194.2ms inference, 1.0ms postprocess per 

Speed: 12.0ms preprocess, 212.0ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 960)

0: 1280x960 1 hold, 204.1ms
Speed: 12.9ms preprocess, 204.1ms inference, 1.0ms postprocess per image at shape (1, 3, 1280, 960)

0: 1280x960 1 hold, 209.6ms
Speed: 11.9ms preprocess, 209.6ms inference, 1.0ms postprocess per image at shape (1, 3, 1280, 960)

0: 1280x960 1 hold, 228.5ms
Speed: 13.7ms preprocess, 228.5ms inference, 1.0ms postprocess per image at shape (1, 3, 1280, 960)

0: 1280x960 7 holds, 244.9ms
Speed: 15.0ms preprocess, 244.9ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 960)

0: 1280x960 5 holds, 232.5ms
Speed: 15.0ms preprocess, 232.5ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 960)

0: 960x1280 9 holds, 227.5ms
Speed: 15.9ms preprocess, 227.5ms inference, 2.0ms postprocess per image at shape (1, 3, 960, 1280)

0: 1280x960 2 holds, 227.0ms
Speed: 14.0ms preprocess, 227.0ms inference, 2.0ms postprocess per image at shape (1, 3, 1280